In [10]:
%pwd

'd:\\Documentation_sample\\research'

In [11]:
import os

os.chdir("D:\Documentation_sample")

In [12]:
%pwd

'D:\\Documentation_sample'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_batch_size : int
    params_valid_size : float
    params_Loss_function : str
    params_learning_rate : float
    params_momentum: float
    params_image_dim : int

In [14]:
from sample_project.constants import *
from sample_project.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_batch_size = self.params.BATCH_SIZE,
            params_valid_size =  self.params.VALID_SIZE,  
            params_Loss_function = self.params.LOSS_FUNCTION,
            params_learning_rate = self.params.LEARNING_RATE,
            params_momentum = self.params.MOMENTUM ,
            params_image_dim = self.params.IMAGE_DIM 
        )

        return prepare_base_model_config

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import torch
import torch.nn as nn

In [17]:
class VGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(nn.Conv2d(1,64,3,1,1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.Conv2d(64,64,3,1,1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2,2),
                                    nn.Conv2d(64,128,3,1,1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.Conv2d(128,128,3,1,1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2,2),
                                    nn.Conv2d(128,256,3,1,1),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(),
                                    nn.Conv2d(256,256,3,1,1),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(),
                                    nn.Conv2d(256,256,3,1,1),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2,2),
                                    nn.Conv2d(256,512,3,1,1),
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(),
                                    nn.Conv2d(512,512,3,1,1),
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(),
                                    nn.Conv2d(512,512,3,1,1),
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2,2),
                                    nn.Flatten(),
                                    nn.Linear(2048,4096),
                                    nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(4096,1028),
                                    nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(1028,10),

                        
                                
        )
    def forward(self,x):
        return self.layers(x)


In [18]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    def get_base_model(self):
        self.model = VGG16()
        dim = self.config.params_image_dim
        print(self.model)


        self.save_model(path=self.config.base_model_path, model=self.model)

    
    @staticmethod
    def save_model(path: Path, model: nn.modules):
        torch.save(model,path)


In [19]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

[2024-07-27 22:52:06,245: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-27 22:52:06,253: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-27 22:52:06,255: INFO: common: created directory at: artifacts]
[2024-07-27 22:52:06,256: INFO: common: created directory at: artifacts/prepare_base_model]
VGG16(
  (layers): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    